In [ ]:
import pandas as pd
import numpy as np
import os
import re
import nltk
import spacy
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from pprint import pprint
from datetime import datetime
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
os.chdir('d:\\Users\\Patri\\Documents\\1.TCC\\Notebooks\\WebScrapAcordaosPDF\\NLTK')

In [ ]:
%%time
df = pd.read_csv('Acordaos2020tratado.csv')

In [ ]:
df.head()

In [ ]:
# Conversão das colunas de texto em string, elimina valores nulos
colunas = ('NomeRelator', 'Ementa', 'Acordao', 'Relatorio', 'Voto', 'VotoVencedor', 'DeclaracaoVoto')
for coluna in colunas: df[coluna] = df[coluna].astype('str')

# Converte a coluna 'Data' para o formato datetime
df['Data'] = pd.to_datetime(df['Data'])

df = df.fillna('')
df = df.replace(np.nan,'')

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# Pesquisar ocorrências da palavra Súmula e variações
pesquisar = ['súmula', 'sumula', 'súmulas', 'sumulas']
pesquisa_str = "|".join(pesquisar)
print (pesquisa_str)

In [ ]:
colunas=['Voto', 'Ementa', 'Relatorio', 'Acordao', 'VotoVencedor', 'DeclaracaoVoto']
df[colunas]

In [ ]:
%%time
# Contar as menções a súmula em cada seção do Acórdão
colunas=['Voto', 'Ementa', 'Relatorio', 'Acordao', 'VotoVencedor', 'DeclaracaoVoto']
resultado = [["Pesquisando:",pesquisa_str],['Total de Documentos',len(df[colunas])]]
for count, colunas in enumerate(colunas):
    resultado.append(['Contém em '+ colunas, len(df[df[colunas].str.contains(pesquisa_str, na=False, case=False)])])
resultado

In [ ]:
%%time
# Seleciona uma coluna para extrair um texto de um acórdão com a palavra súmula
colunas=['Voto', 'Ementa', 'Relatorio', 'Acordao', 'VotoVencedor', 'DeclaracaoVoto']
pesquisar = ['súmula', 'sumula', 'súmulas', 'sumulas']
pesquisa_str = "|".join(pesquisar)

for count, coluna in enumerate(colunas): print (count, coluna)

coluna_escolhida = colunas[5]
df_sumula = df[df[coluna_escolhida].str.contains(pesquisa_str, na=False, case=False)]
df_sumula_coluna = df_sumula[colunas]
display(df_sumula_coluna)

In [ ]:
# Recebe um texto ou series e retorna os tokens
def tokeniza(texto):
    if type(texto) == "Series":
        acordao_text = texto.to_string()
    else:
        acordao_text = str(texto)
    acordao_text = acordao_text.lower() # coloca tudo em minúsculo
    acordao_text = re.sub('(º)', ' ', acordao_text) # remove 'º' substituindo por espaço
    acordao_words = RegexpTokenizer(r'\w+') # remove pontuação ao selecionar apenas palavras
    acordao_tokens = acordao_words.tokenize(acordao_text) # tokeniza (separa as palavras)

    # Extrair Stopwords em Português
    pt_stops = set(stopwords.words('portuguese'))
    pt_stops.update(['fls', 'cuida', 'ª', 'n', 'fl'])
    words = (acordao_tokens)
    lista_words = [word for word in words if word not in pt_stops] # remove stopwords
    if not len(lista_words): lista_words = []
    return lista_words 

In [ ]:
%%time
# Tokeniza trechos do acórdão e salva tokens em uma nova coluna
colunas=['Voto', 'Ementa', 'Relatorio', 'Acordao', 'VotoVencedor', 'DeclaracaoVoto']

for coluna in colunas:
    df['t_'+coluna] = df[coluna].apply(tokeniza)

In [ ]:
# Identifica ocorrências da palavra "Súmula" e informações adicionais ao redor
# Recebe um texto tokenizado e retorna um dataframe
def localiza_sumula(texto):

    # Tokeniza texto, remove stopwords e pontuação
    listOfTokens = texto
       
    # Estabelece variações da palavra 'súmula'
    words2find = ['súmula', 'sumula', 'súmulas', 'sumulas']
    context = 4

    # Encontrar ocorrências da string de pesquisa e mostrar as palavras em torno
    # https://github.com/sgsinclair/alta/blob/a482d343142cba12030fea4be8f96fb77579b3ab/ipynb/utilities/Concordances.ipynb

    def makeConc(words2conc,list2FindIn,context2Use):
        end = len(list2FindIn)

        # Nomear as colunas das informações da súmula
        col_conteudo = "Conteudo"
        col_posicao = "Posicao"
        col_num_sum = "Sumula n"
        col_tribunal = "Tribunal"
        col_vinculante = "Vinculacao"
        col_aplica = "Aplicabilidade"
        colunas_sumulas = (col_conteudo, col_posicao, col_num_sum, col_tribunal, col_vinculante, col_aplica)

        concDF = pd.DataFrame(columns = colunas_sumulas)

        for location in range(end):
            for word2conc in words2conc:
                if list2FindIn[location] == word2conc:
                    if (location - context2Use) < 0:
                        beginCon = 0
                    else:
                        beginCon = location - context2Use

                    if (location + context2Use) > end:
                        endCon = end
                    else:
                        endCon = location + context2Use

                    theContext = (list2FindIn[location:endCon])
                    concordanceLine = ' '.join(theContext)
                                      
                    # Encontrar as informações da súmula
                    txt_conteudo = concordanceLine
                    txt_posicao = str(location)
                    txt_num_sum = ""
                    txt_tribunal = ""
                    txt_vinculante = ""
                    txt_aplica = ""
                    for location_number in range (beginCon,endCon):
                        achou = list2FindIn[location_number]
                        tribunais = ["carf", "stf", "stj", "supremo", "tst", "trt"]
                        inaplicaveis = ["inaplicabilidade", "não", "inaplicável", "incabível", "inaplica"]
                        if achou.isnumeric() and location_number > location and int(achou) <= 161 : txt_num_sum = int(achou)
                        if achou in tribunais    and achou not in txt_tribunal: txt_tribunal = achou
                        if achou == "vinculante" and achou not in txt_vinculante: txt_vinculante = achou
                        if achou in inaplicaveis and achou not in txt_aplica: txt_aplica = achou
                    
                    nova_linha = {
                        col_conteudo   : txt_conteudo,
                        col_posicao    : txt_posicao,
                        col_num_sum    : txt_num_sum,
                        col_tribunal   : txt_tribunal,
                        col_vinculante : txt_vinculante,
                        col_aplica     : txt_aplica
                    }                        
                    if bool(txt_num_sum) : concDF = concDF.append(nova_linha, ignore_index=True)

        return concDF

    concDF2 = makeConc(words2find,listOfTokens,int(context))

    return concDF2

In [ ]:
# Identifica o número das súmulas CARF aplicáveis
# Recebe texto tokenizado (constante nas colunas t_secao)
# Retorna lista de números
def identifica_sumulas(t_coluna):
    if not len(t_coluna):
        return []
    else:
        df_sumula = localiza_sumula(t_coluna)
        # Filtrar apenas Súmulas do CARF aplicáveis    
        df_carf = df_sumula[['Tribunal','Sumula n','Aplicabilidade']]
        df_carf = df_carf[(df_carf['Tribunal'] == 'carf') & (df_carf['Aplicabilidade'] == '')]
        df_carf = df_carf.drop_duplicates()
        df_carf = df_carf['Sumula n'].sort_values().tolist()
        if len(df_carf):
            return df_carf

In [ ]:
%%time
#
# Função mais demorada desse notebook!!!!!
#
# Aplica a função de listar em uma nova coluna o número das súmulas de cada seção do acordão:
# Utiliza o texto tokeniza de cada seção (voto, ementa, relatório, acordão, voto vencedor e declaração de voto), 
# localiza nele as ocorrências da palavra súmula e suas variações, extrai informações adicionais (tribunal, aplicabilidade, número)
# e retorna uma lista com os números das súmulas CARF aplicadas naquele texto. O resultado é inserido em uma nova coluna no dataframe.
# Entrada: colunas t_seção
# Saída: colunas s_seção

colunas=['Voto', 'Ementa', 'Relatorio', 'Acordao', 'VotoVencedor', 'DeclaracaoVoto']
# colunas=['DeclaracaoVoto']

for coluna in colunas:
    df['s_'+coluna] = df['t_'+coluna].apply(identifica_sumulas)

In [ ]:
%%time
df.to_csv('Acordaos2020sumulado.csv', index=False)